# Filter Variants

This comes from the example notebook and the [tour of scikit-allel](http://alimanfoo.github.io/2016/06/10/scikit-allel-tour.html) blog post.

There are many possible approaches to filtering variants. The simplest approach is define thresholds on variant attributes like DP, MQ and QD, and exclude SNPs that fall outside of a defined range (a.k.a. “hard filtering”). This is crude but simple to implement and in many cases may suffice, at least for an initial exploration of the data.

In [2]:
import numpy as np
import zarr
import pandas as pd
import dask.array as da
import allel
import scipy
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('notebook')
%matplotlib inline

In [3]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

distributed.scheduler - INFO - Clear task state
/opt/conda/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
distributed.scheduler - INFO -   Scheduler at:   tcp://10.35.63.92:42375
distributed.scheduler - INFO -   dashboard at:                    :39099


## Import the Variant Data

In [5]:
import gcsfs

gcs_bucket_fs = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='anon', access='read_only')

storage_path = 'ag1000g-release/phase2.AR1/variation/main/zarr/pass/ag1000g.phase2.ar1.pass'
store = gcsfs.mapping.GCSMap(storage_path, gcs=gcs_bucket_fs, check=False, create=False)
callset = zarr.Group(store)

In [6]:
chrom = '3L'
variants = allel.VariantChunkedTable(callset[chrom]['variants'], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD'],
                                     index='POS')
variants

<VariantChunkedTable shape=(10640388,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1', (3,)), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4')]
   nbytes=202.9M cbytes=-6 cratio=-35467960.0
   values=zarr.hierarchy.Group>

## Filter using a Hard Expression

Define the hard filter using an expression. This is just a string of Python code, which we will evaluate in a moment.

In [7]:
filter_expression = '(QD > 5) & (MQ > 40) & (DP > 15000) & (DP < 30000)'

In [8]:
variant_selection = variants.eval(filter_expression)[:]
variant_selection

array([False, False, False, ..., False, False, False])

How many variants do we want to keep?

In [10]:
# Number of variants we're keeping based on filter_expression criteria
np.count_nonzero(variant_selection)

304050

How many variants do we filter out?

In [12]:
# Number of variants we're tossing based on filter_expression criteria
np.count_nonzero(~variant_selection)

10336338

distributed.scheduler - INFO - Register tcp://10.35.87.2:38715
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.87.2:38715
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.67.2:36717
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.67.2:36717
distributed.core - INFO - Starting established connection


Now that we have our variant filter, let’s make a new variants table with only rows for variants that pass our filter.

In [14]:
variants_pass = variants.compress(variant_selection)
variants_pass

distributed.scheduler - INFO - Register tcp://10.35.93.2:44567
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.93.2:44567
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.92.2:35361
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.92.2:35361
distributed.core - INFO - Starting established connection


<VariantChunkedTable shape=(304050,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1', (3,)), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4')]
   nbytes=5.8M cbytes=3.3M cratio=1.7
   values=allel.chunked.storage_zarr.ZarrTable>

distributed.scheduler - INFO - Register tcp://10.35.69.2:34855
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.69.2:34855
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.85.2:36335
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.85.2:36335
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.68.2:46027
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.68.2:46027
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.94.2:35005
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.94.2:35005
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.76.2:40437
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.76.2:40437
distributed.core - INFO